In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('Bigmart_preprocessed_final.csv')

In [3]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_MRP2
0,1,9.300,0,0.016047,4,249.8092,9,1999,1,0,1,3735.1380,3
1,0,5.920,1,0.019278,14,48.2692,3,2009,1,2,2,443.4228,0
2,1,17.500,0,0.016760,10,141.6180,9,1999,1,0,1,2097.2700,2
3,1,13.650,1,0.012741,13,57.6588,1,1987,0,2,1,343.5528,0
4,1,12.600,0,0.127470,13,107.7622,5,1985,1,2,3,4022.7636,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810,1,6.865,0,0.056783,13,214.5218,1,1987,0,2,1,2778.3834,3
7811,1,8.380,1,0.046982,0,108.1570,7,2002,1,1,1,549.2850,1
7812,2,10.600,0,0.035186,8,85.1224,6,2004,2,1,1,1193.1136,1
7813,1,7.210,1,0.145221,13,103.1332,3,2009,1,2,2,1845.5976,1


In [4]:
df.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_MRP2
0,1,9.30,0,0.016047,4,249.8092,9,1999,1,0,1,3735.1380,3
1,0,5.92,1,0.019278,14,48.2692,3,2009,1,2,2,443.4228,0
2,1,17.50,0,0.016760,10,141.6180,9,1999,1,0,1,2097.2700,2
3,1,13.65,1,0.012741,13,57.6588,1,1987,0,2,1,343.5528,0
4,1,12.60,0,0.127470,13,107.7622,5,1985,1,2,3,4022.7636,1


In [5]:
pip install sweetviz

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sweetviz as sv
eda = sv.analyze(df)
eda.show_html('edal.html')

                                             |                                             | [  0%]   00:00 ->…

Report edal.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
from scipy import stats


num=['Item_Weight',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year',
 'Item_Outlet_Sales']
z_scores = np.abs(stats.zscore(df[num]))


z_score_threshold = 3

# Find the indices of outliers based on the threshold
outlier_indices = np.where(z_scores > z_score_threshold)[0]

# Remove the outliers from the training data
df = df.drop(df.index[outlier_indices])

In [8]:
df.shape

(7742, 13)

# model trainning

In [9]:
X=df.drop(['Item_Outlet_Sales','Outlet_Establishment_Year'],axis=1)
y=df['Item_Outlet_Sales']

################################
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,# input data
                                                y,#output data
                                                random_state=42,#it selects the random samples
                                                test_size=0.30)#test data splitting from original data 30%==03===30%

from sklearn.ensemble import GradientBoostingRegressor

GR=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=3)

GR.fit(X_train,y_train)


y_pred_GR=GR.predict(X_test)


from sklearn.metrics import r2_score,mean_squared_error

R2_GR=r2_score(y_test,y_pred_GR)
MSE_GR=mean_squared_error(y_test,y_pred_GR)
RMSE_GR=np.sqrt(MSE_GR)


print("rsquare_GR:",R2_GR)
print("mse_GR:", MSE_GR)
print("rmse_GR:",RMSE_GR)


rsquare_GR: 0.5802014401676308
mse_GR: 961982.4823416432
rmse_GR: 980.8070566332826


In [10]:
#pickle file
import pickle
pickle.dump(GR,
            open('model.pkl','wb'))


In [11]:
# Loading model to compare the results
model= pickle.load(open('model.pkl','rb'))

In [12]:
X.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type', 'Item_MRP2'],
      dtype='object')

In [14]:
model.predict([[1,6.7,2,0.07,1,235,1,1,1,1,1]])

C:\Users\Saikiran\New folder\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


array([4027.0383621])

In [15]:
y_pred_GR

array([ 943.70964248, 3050.13013568, 2284.12995018, ..., 3952.72526261,
       1113.54684293, 2584.40855471])